<a href="https://colab.research.google.com/github/mudogruer/LLMs/blob/main/microsoft_phi_2_zero_shot_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q -U accelerate bitsandbytes bert-score

In [ ]:
from transformers import BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_quanty_type = "nf4",
    bnb_4bit_use_double_quanty = True,
)


In [ ]:
# from huggingface_hub import login
# login()

In [ ]:
model_name = "microsoft/phi-2"

from transformers import AutoTokenizer
import torch

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_4bit=True,
    quantization_config=bnb_config,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    trust_remote_code = True,

)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
from transformers import pipeline

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer = tokenizer,
    torch_dtype = torch.bfloat16,
    device_map = "auto"

)


2024-02-19 21:49:14.877245: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-19 21:49:14.877301: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-19 21:49:14.878867: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [ ]:
prompt = "Instruct:"+"What is the heaviest element?"+"\nOutput:"

sequence = pipe(
    prompt,
    do_sample = True,
    max_new_tokens = 100,
    temperature = 0.2,
    top_k = 2,
    top_p = 1.0,
)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [ ]:
print(sequence[0]['generated_text'])

Instruct:What is the heaviest element?
Output: The heaviest element is oganesson.



In [ ]:
sequence

[{'generated_text': 'Instruct:What is the heaviest element?\nOutput: The heaviest element is oganesson.\n'}]

In [ ]:
prompt = "Instruct:"+"What is the gravity on mars?"+"\nOutput:"

sequence = pipe(
    prompt,
    do_sample = True,
    max_new_tokens = 100,
    temperature = 0.2,
    top_k = 2,
    top_p = 1.0,
)
print(sequence[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Instruct:What is the gravity on mars?
Output: What is the gravity on mars?



In [ ]:
def Sequence(promt):
    sequence = pipe(
    prompt,
    do_sample = True,
    max_new_tokens = 30,
    temperature = 0.1,
    top_k = 2,
    top_p = 1.0,
    pad_token_id=tokenizer.eos_token_id
    )
    return sequence

In [ ]:
prompt = "Instruct:"+"Who invented mirror telescope?"+"\nOutput:"

print(Sequence(prompt)[0]['generated_text'])

Instruct:Who invented mirror telescope?
Output: The mirror telescope was invented by Galileo Galilei in 1609.



In [ ]:
from datasets import load_dataset

dataset = load_dataset("sciq")

  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['question', 'distractor3', 'distractor1', 'distractor2', 'correct_answer', 'support'],
        num_rows: 11679
    })
    validation: Dataset({
        features: ['question', 'distractor3', 'distractor1', 'distractor2', 'correct_answer', 'support'],
        num_rows: 1000
    })
    test: Dataset({
        features: ['question', 'distractor3', 'distractor1', 'distractor2', 'correct_answer', 'support'],
        num_rows: 1000
    })
})

In [ ]:
dataset_validation = load_dataset('sciq',split='validation[:20%]') #first %20 of validation data

In [ ]:
dataset_validation

Dataset({
    features: ['question', 'distractor3', 'distractor1', 'distractor2', 'correct_answer', 'support'],
    num_rows: 200
})

In [ ]:
answers = []
for question in dataset_validation["question"]:
    prompt = "Instruct:"+question+"\nOutput:"
    output_sequences = Sequence(prompt)[0]['generated_text']
    question_and_answer = output_sequences.split("\nOutput:")
    answer = question_and_answer[1]
    answers.append(answer)

/opt/conda/lib/python3.10/site-packages/transformers/pipelines/base.py:1123: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


In [ ]:
from bert_score import score

# Assuming `answers` is your list of generated answers and `actual_answers` is the list of reference answers
# Both should be lists of strings

# Calculate BertScore
P, R, F1 = score(answers, dataset_validation["correct_answer"], lang='en', verbose=True)

# `P`, `R`, and `F1` are returned as tensors, so you might want to convert them to Python lists or take their mean
bertscore_precision = P.mean().item()
bertscore_recall = R.mean().item()
bertscore_f1 = F1.mean().item()

print(f"Precision: {bertscore_precision:.3f}")
print(f"Recall: {bertscore_recall:.3f}")
print(f"F1: {bertscore_f1:.3f}")


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


calculating scores...
computing bert embedding.


  0%|          | 0/7 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/4 [00:00<?, ?it/s]

done in 1.30 seconds, 153.89 sentences/sec
Precision: 0.825
Recall: 0.840
F1: 0.832


In [ ]:
import pandas as pd
df = pd.DataFrame({"generated_answers":answers,"correct_answers":dataset_validation["correct_answer"]})

In [ ]:
df.head(10)

,generated_answers,correct_answers
0,Charles Darwin\n,darwin
1,Each specific polypeptide has a unique linear...,amino
2,A frameshift mutation is a deletion or inserti...,nucleotides
3,A wetland.\n,wetland
4,The sun.\n,the sun
5,"Arteries, veins, and capillaries are examples...",blood vessels
6,The two general categories of biochemical rea...,catabolic and anabolic
7,The clay fractions of soils derived from gran...,volcanic ash
8,The human brain.\n,brain
9,"The term ""knot"" can refer to a rope in a part...",lariat
